In [1]:
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import GlobalAveragePooling2D
from keras.layers import Dense
from keras.layers import Input, Dense
from keras.models import Model
from keras.layers import Flatten
from keras.layers import merge
from keras.layers import Reshape
from keras.layers import Activation
from keras.layers import concatenate
from keras.layers import Concatenate

from ssd_layer import DefaultBox

import keras.backend as K
import tensorflow as tf

C:\Users\Ryoji Nomura\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# define IO shapes
img_size = (300, 300, 3)
num_classes = 3

In [3]:
## root model
net = {}

# Input
net['inputs'] = Input(shape=img_size, name='input')

# Block 1
net['conv1_1'] = Conv2D(64, (3, 3),activation='relu',padding='same',name='conv1_1')(net['inputs'])
net['conv1_2'] = Conv2D(64, (3, 3),activation='relu',padding='same',name='conv1_2')(net['conv1_1'])
net['pool1'] = MaxPooling2D((2,2),strides=(2,2),padding='same',name='pool1')(net['conv1_2'])

# # Block 2
net['conv2_1'] = Conv2D(128, (3, 3),activation='relu',padding='same',name='conv2_1')(net['pool1'])
net['conv2_2'] = Conv2D(128, (3, 3),activation='relu',padding='same',name='conv2_2')(net['conv2_1'])
net['pool2'] = MaxPooling2D((2, 2), strides=(2, 2), padding='same',name='pool2')(net['conv2_2'])

# # FC3
net['fc3'] = Conv2D(1024, (3, 3), dilation_rate=(6, 6),activation='relu', padding='same',name='fc3')(net['pool2'])

# # FC4
net['fc4'] = Conv2D(1024, (1, 1), activation='relu',padding='same',name='fc4')(net['fc3'])

# # Block 5
net['conv5_1'] = Conv2D(128, (3, 3),activation='relu',padding='same',name='conv5_1')(net['fc4'])
net['pool5'] = MaxPooling2D((2, 2), strides=(2, 2), padding='same',name='pool5')(net['conv5_1'])

# # Block 6
net['conv6_1'] = Conv2D(128, (3, 3),activation='relu',padding='same',name='conv6_1')(net['pool5'])
net['pool6'] = MaxPooling2D((2, 2), strides=(2, 2), padding='same',name='pool6')(net['conv6_1'])

# # Last Pool
net['pool7'] = GlobalAveragePooling2D(name='pool7')(net['pool6'])
net['dence8'] = Dense(num_classes, activation='softmax', name='dence8')(net['pool7'] )

model = Model(net['inputs'], net['dence8'])

In [4]:
#model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           (None, 300, 300, 3)       0         
_________________________________________________________________
conv1_1 (Conv2D)             (None, 300, 300, 64)      1792      
_________________________________________________________________
conv1_2 (Conv2D)             (None, 300, 300, 64)      36928     
_________________________________________________________________
pool1 (MaxPooling2D)         (None, 150, 150, 64)      0         
_________________________________________________________________
conv2_1 (Conv2D)             (None, 150, 150, 128)     73856     
_________________________________________________________________
conv2_2 (Conv2D)             (None, 150, 150, 128)     147584    
_________________________________________________________________
pool2 (MaxPooling2D)         (None, 75, 75, 128)       0         
__________

In [5]:
## branch model for feature map
dim_box = 4
num_def = 6

net['conv5_1_mbox_loc'] = Conv2D(num_def*dim_box,(3,3),padding='same',name='conv5_1_mbox_loc')(net['conv5_1'])
net['conv5_1_mbox_loc_flat'] = Flatten(name='conv5_1_mbox_loc_flat')(net['conv5_1_mbox_loc'])

net['conv5_1_mbox_conf'] = Conv2D(num_def*num_classes,(3,3),padding='same',name='conv5_1_mbox_conf')(net['conv5_1'])
net['conv5_1_mbox_conf_flat'] = Flatten(name='conv5_1_mbox_conf_flat')(net['conv5_1_mbox_conf'])

net['pool6_mbox_loc'] = Conv2D(num_def*dim_box,(3,3),padding='same',name='pool6_mbox_loc')(net['pool6'])
net['pool6_mbox_loc_flat'] = Flatten(name='pool6_mbox_loc_flat')(net['pool6_mbox_loc'])

net['pool6_mbox_conf'] = Conv2D(num_def*num_classes,(3,3),padding='same',name='pool6_mbox_conf')(net['pool6'])
net['pool6_mbox_conf_flat'] = Flatten(name='pool6_mbox_conf_flat')(net['pool6_mbox_conf'])

net['mbox_loc'] = concatenate([net['conv5_1_mbox_loc_flat'], net['pool6_mbox_loc_flat']], name='mbox_loc')
net['mbox_conf'] = concatenate([net['conv5_1_mbox_conf_flat'],net['pool6_mbox_conf_flat']],name='mbox_conf')

net['mbox_loc'] = Reshape((-1, dim_box),name='mbox_loc_final')(net['mbox_loc'])
net['mbox_conf'] = Reshape((-1, num_classes),name='mbox_conf_logits')(net['mbox_conf'])
net['mbox_conf'] = Activation('softmax',name='mbox_conf_final')(net['mbox_conf'])
net['predictions'] = concatenate([net['mbox_loc'], net['mbox_conf']], name='predictions')

model_feature_map = Model(net['inputs'], net['predictions'])

In [6]:
#model_feature_map.summary()

In [12]:
## branch model for feature map

net['conv5_1_mbox_loc'] = Conv2D(num_def*dim_box,(3,3),padding='same',name='conv5_1_mbox_loc')(net['conv5_1'])
net['conv5_1_mbox_loc_flat'] = Flatten(name='conv5_1_mbox_loc_flat')(net['conv5_1_mbox_loc'])
print(net['conv5_1_mbox_loc_flat'])

net['conv5_1_mbox_conf'] = Conv2D(num_def*num_classes,(3,3),padding='same',name='conv5_1_mbox_conf')(net['conv5_1'])
net['conv5_1_mbox_conf_flat'] = Flatten(name='conv5_1_mbox_conf_flat')(net['conv5_1_mbox_conf'])
print(net['conv5_1_mbox_conf_flat'])

net['conv5_1_mbox_defbox'] = DefaultBox(img_size, 20, 30, aspect_ratios=[2, 3],variances=[0.1, 0.1, 0.1, 0.1],name='conv5_1_mbox_defbox')(net['conv5_1'])
#net['conv5_1_mbox_defbox_flat'] = Flatten(name='conv5_1_mbox_defbox')(net['conv5_1_mbox_defbox'])
#net['conv5_1_mbox_defbox'] = Reshape((-1,num_boxes, 8))(net['conv5_1_mbox_defbox'])
#net['conv5_1_mbox_defbox'] = K.tile(net['conv5_1_mbox_defbox'],(1,1,1))
print(net['conv5_1_mbox_defbox'])

net['pool6_mbox_loc'] = Conv2D(num_def*dim_box,(3,3),padding='same',name='pool6_mbox_loc')(net['pool6'])
net['pool6_mbox_loc_flat'] = Flatten(name='pool6_mbox_loc_flat')(net['pool6_mbox_loc'])
print(net['pool6_mbox_loc_flat'])

net['pool6_mbox_conf'] = Conv2D(num_def*num_classes,(3,3),padding='same',name='pool6_mbox_conf')(net['pool6'])
net['pool6_mbox_conf_flat'] = Flatten(name='pool6_mbox_conf_flat')(net['pool6_mbox_conf'])
print(net['pool6_mbox_conf_flat'])

net['pool6_mbox_defbox'] = DefaultBox(img_size, 45, 60, aspect_ratios=[2, 3],variances=[0.1, 0.1, 0.1, 0.1],name='pool6_mbox_defbox')(net['pool6'])
#net['pool6_mbox_defbox_flat'] = Flatten(name='pool6_mbox_defbox_flat')(net['pool6_mbox_defbox'])
#net['pool6_mbox_defbox'] = Reshape((-1, 8))(net['pool6_mbox_defbox'])
#net['pool6_mbox_defbox'] = K.tile(net['pool6_mbox_defbox'],(1,1,1))
print(net['pool6_mbox_defbox'])

net['mbox_loc'] = concatenate([net['conv5_1_mbox_loc_flat'], net['pool6_mbox_loc_flat']], name='mbox_loc', axis=1)
num_boxes = net['mbox_loc']._keras_shape[-1] // 4
net['mbox_loc'] = Reshape((num_boxes, dim_box),name='mbox_loc_final')(net['mbox_loc'])
print(net['mbox_loc'])

net['mbox_conf'] = concatenate([net['conv5_1_mbox_conf_flat'],net['pool6_mbox_conf_flat']],name='mbox_conf', axis=1)
net['mbox_conf'] = Reshape((num_boxes, num_classes),name='mbox_conf_logits')(net['mbox_conf'])
net['mbox_conf'] = Activation('softmax',name='mbox_conf_final')(net['mbox_conf'])
print(net['mbox_conf'])

net['mbox_defbox'] = concatenate([net['conv5_1_mbox_defbox'],net['pool6_mbox_defbox']],name='mbox_defbox',axis=1)
print(net['mbox_defbox'])

#net['predictions'] = concatenate([net['mbox_loc'], net['mbox_conf'], net['mbox_defbox']], name='predictions',axis=2)
net['predictions'] = concatenate([net['mbox_loc'], net['mbox_conf'], net['mbox_defbox']], name='predictions',axis=2)
print(net['predictions'])

model_pred = Model(net['inputs'], net['predictions'])

Tensor("conv5_1_mbox_loc_flat_2/Reshape:0", shape=(?, ?), dtype=float32)
Tensor("conv5_1_mbox_conf_flat_2/Reshape:0", shape=(?, ?), dtype=float32)
Tensor("conv5_1_mbox_defbox_1/Tile:0", shape=(?, 33750, 8), dtype=float32)
Tensor("pool6_mbox_loc_flat_2/Reshape:0", shape=(?, ?), dtype=float32)
Tensor("pool6_mbox_conf_flat_2/Reshape:0", shape=(?, ?), dtype=float32)
Tensor("pool6_mbox_defbox_1/Tile:0", shape=(?, 2166, 8), dtype=float32)
Tensor("mbox_loc_final_2/Reshape:0", shape=(?, 35916, 4), dtype=float32)
Tensor("mbox_conf_final_2/truediv:0", shape=(?, 35916, 3), dtype=float32)
Tensor("mbox_defbox_1/concat:0", shape=(?, 35916, 8), dtype=float32)
Tensor("predictions_2/concat:0", shape=(?, 35916, 15), dtype=float32)


In [13]:
model_pred.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              (None, 300, 300, 3)  0                                            
__________________________________________________________________________________________________
conv1_1 (Conv2D)                (None, 300, 300, 64) 1792        input[0][0]                      
__________________________________________________________________________________________________
conv1_2 (Conv2D)                (None, 300, 300, 64) 36928       conv1_1[0][0]                    
__________________________________________________________________________________________________
pool1 (MaxPooling2D)            (None, 150, 150, 64) 0           conv1_2[0][0]                    
__________________________________________________________________________________________________
conv2_1 (C